In [1]:
import sys
sys.path.append('..')
import numpy as np

In [2]:
from dset_helpers import bool_to_bin,data_given_param

#### We will use sweep rate of 15 MHz so that the state prep is "as adiabatic as possible"... In theory this should be the best data.

In [3]:
data_16x16_15MHz_per_us = np.load('../../KZ_Data/KZ_data_16x16_15_MHz_per_us.npz')

In [4]:
file_list_15 = data_16x16_15MHz_per_us.files

In [5]:
for file in file_list_15:
    print(file)
    print(data_16x16_15MHz_per_us[file])

params
[-1.77948113 -1.6615566  -1.54363208 -1.42570755 -1.30778302 -1.18985849
 -1.07193396 -0.95400943 -0.83608491 -0.71816038 -0.60023585 -0.48231132
 -0.36438679 -0.24646226 -0.12853774 -0.01061321  0.10731132  0.22523585
  0.34316038  0.46108491  0.57900943  0.69693396  0.81485849  0.93278302
  1.05070755  1.16863208  1.2865566   1.40448113  1.52240566  1.64033019
  1.75825472  1.87617925  1.99410377  2.1120283   2.22995283  2.34787736
  2.46580189  2.58372642  2.70165094  2.81957547  2.9375      3.05542453
  3.17334906  3.29127358  3.40919811  3.52712264  3.64504717  3.7629717 ]
rydberg_data
[[[[False False False ... False False False]
   [False  True False ... False False False]
   [False False False ... False False False]
   ...
   [ True  True  True ...  True  True  True]
   [ True False False ...  True  True  True]
   [ True  True  True ...  True  True  True]]

  [[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... 

#### Choose the right "params" and associated data

The "params" file contains values for $\frac{\delta}{\Omega}$ 

In [6]:
print(data_16x16_15MHz_per_us['params'])

[-1.77948113 -1.6615566  -1.54363208 -1.42570755 -1.30778302 -1.18985849
 -1.07193396 -0.95400943 -0.83608491 -0.71816038 -0.60023585 -0.48231132
 -0.36438679 -0.24646226 -0.12853774 -0.01061321  0.10731132  0.22523585
  0.34316038  0.46108491  0.57900943  0.69693396  0.81485849  0.93278302
  1.05070755  1.16863208  1.2865566   1.40448113  1.52240566  1.64033019
  1.75825472  1.87617925  1.99410377  2.1120283   2.22995283  2.34787736
  2.46580189  2.58372642  2.70165094  2.81957547  2.9375      3.05542453
  3.17334906  3.29127358  3.40919811  3.52712264  3.64504717  3.7629717 ]


Note that the value for $\Omega$ is also given, allowing us to determine $\delta$.

In [7]:
print(data_16x16_15MHz_per_us['rabi_freq'])

4.24


In [8]:
deltas = data_16x16_15MHz_per_us['params']*data_16x16_15MHz_per_us['rabi_freq']

In [9]:
print(deltas)

[-7.545 -7.045 -6.545 -6.045 -5.545 -5.045 -4.545 -4.045 -3.545 -3.045
 -2.545 -2.045 -1.545 -1.045 -0.545 -0.045  0.455  0.955  1.455  1.955
  2.455  2.955  3.455  3.955  4.455  4.955  5.455  5.955  6.455  6.955
  7.455  7.955  8.455  8.955  9.455  9.955 10.455 10.955 11.455 11.955
 12.455 12.955 13.455 13.955 14.455 14.955 15.455 15.955]


There is also a value called "rydberg_resonance" and we do not know what this means. I think it is some intrinsic resonance in the Rydberg atoms that compete with the lab-controlled delta. Notice what happens when you add this rydberg resonance to the deltas above.

In [10]:
lab_deltas = deltas+data_16x16_15MHz_per_us['ryd_resonance']

In [11]:
print(lab_deltas)

[-8.  -7.5 -7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5
 -1.  -0.5  0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5
  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5
 13.  13.5 14.  14.5 15.  15.5]


These are nice clean numbers, so I think this is the lab-controlled detuning and then they take into account (subtract out) the rydberg resonance which competes with delta. The "net delta" is the one that comes directly from params... we think...

# The paper gives $\frac{\delta_c}{\Omega} = 1.12(4)$ !

So we want to take some critical delta (as close to this as possible) and then two values that are on either side of this critical point (considerably far away to be clearly in the "ordered/disordered phases".

Looking at the values above, we can see that there are two values of $\frac{\delta}{\Omega}$ close to the critical point and they are:

In [12]:
print(data_16x16_15MHz_per_us['params'][24],' and ',data_16x16_15MHz_per_us['params'][25])

1.0507075471698113  and  1.1686320754716981


The associated values of delta are:

In [13]:
delta_midlow = data_16x16_15MHz_per_us['params'][24]*data_16x16_15MHz_per_us['rabi_freq']
delta_midhigh = data_16x16_15MHz_per_us['params'][25]*data_16x16_15MHz_per_us['rabi_freq']
print(delta_midlow,' and ',delta_midhigh)

4.455  and  4.955


It is worth noting that the this slower sweep rate has extra values in params (48 vs 24) and so there is no data corresponding to the second param above for the other sweep rates! For the values of params that we choose for the ordered and disordered phases, it might be worth choosing values of $\frac{\delta}{\Omega}$ that appear in the data for other sweep rates too. At the end of the day it probably doesnt matter too much.

For the lower param value:

In [14]:
print(data_16x16_15MHz_per_us['params'][12])

-0.3643867924528302


The associated value of delta is:

In [15]:
delta_low = data_16x16_15MHz_per_us['params'][12]*data_16x16_15MHz_per_us['rabi_freq']
print(delta_low)

-1.545


And for the higher param value:

In [16]:
print(data_16x16_15MHz_per_us['params'][42])

3.1733490566037736


The associated value of delta is:

In [17]:
print(data_16x16_15MHz_per_us['params'][42]*data_16x16_15MHz_per_us['rabi_freq'])

13.455


Now lets save these values:

In [59]:
low_param = data_16x16_15MHz_per_us['params'][12]
midlow_param = data_16x16_15MHz_per_us['params'][24]
midhigh_param = data_16x16_15MHz_per_us['params'][25]
high_param = data_16x16_15MHz_per_us['params'][42]
highest = data_16x16_15MHz_per_us['params'][47]

# To reiterate, our values of $\frac{\delta}{\Omega}$ are: -0.36438, 1.05070, 1.16863, and 3.17334

This means 

#### Now we want to isolate the data associated with these parameters

In [19]:
low_param_index = np.where(data_16x16_15MHz_per_us['params']==low_param)[0]

In [20]:
print("This low value of param ")

This low value of param 


In [21]:
data_low_param = data_16x16_15MHz_per_us['rydberg_data'][:,:,low_param_index,:]
data_low_param = bool_to_bin(data_low_param)

In [22]:
print(np.shape(data_low_param))

(16, 16, 1, 1000)


In [23]:
midlow_param_index = np.where(data_16x16_15MHz_per_us['params']==midlow_param)[0]

In [24]:
data_midlow_param = data_16x16_15MHz_per_us['rydberg_data'][:,:,midlow_param_index,:]
data_midlow_param = bool_to_bin(data_midlow_param)

In [25]:
print(np.shape(data_midlow_param))

(16, 16, 1, 1000)


In [26]:
midhigh_param_index = np.where(data_16x16_15MHz_per_us['params']==midhigh_param)[0]

In [27]:
data_midhigh_param = data_16x16_15MHz_per_us['rydberg_data'][:,:,midhigh_param_index,:]
data_midhigh_param = bool_to_bin(data_midhigh_param)

In [28]:
print(np.shape(data_midhigh_param))

(16, 16, 1, 1000)


In [61]:
high_param_index = np.where(data_16x16_15MHz_per_us['params']==high_param)[0]

In [62]:
data_high_param = data_16x16_15MHz_per_us['rydberg_data'][:,:,high_param_index,:]
data_high_param = bool_to_bin(data_high_param)

In [63]:
print(np.shape(data_high_param))

(16, 16, 1, 1000)


So now we have 16x16 data for each of these four values of $\frac{\delta}{\Omega}$! Each experiment was run 1000 times, so essentially we have 1000 shots for each of these sets of parameters.

In [70]:
data_highest = data_16x16_15MHz_per_us['rydberg_data'][:,:,-1,:]
data_highest= bool_to_bin(data_highest)
print(np.shape(data_highest))

(16, 16, 1000)


## Write script to do all of the above with one call

something broke here, not sure what... I think I maybe changed the directory my script looks in to get the data.

In [32]:
data_test_high = data_given_param(15,3.17334906)

This is not a valid value of delta!


print(np.mean(abs(data_test_high-data_high_param)))

which means the above datasets match! Yay!

# Look at actual rydberg data for each value of delta

Calculate stag mag for these datasets

In [33]:
import sys
sys.path.append('..')
from stag_mag import coord_to_site,generate_sublattices_square,calculate_stag_mag

### Mean occupation

In [34]:
print(np.mean(data_low_param))

0.14557421875


low because in the disordered regime where energy is penalized if a rydberg atom is occupied

In [35]:
print(np.mean(data_high_param))

0.47578515625


close to 1/2 because in the checkerboard regime

In [36]:
print(np.mean(data_midlow_param))

0.295046875


In [37]:
print(np.mean(data_midhigh_param))

0.30913671875


this is consistent

### Stag Mag

In [46]:
print(np.shape(data_low_param))
print(np.sum(data_low_param[:,:,:,119]))

(16, 16, 1, 1000)
30


Need to reshape to be in correct format for stag mag calculation. Should be (nsamples, natoms)

In [53]:
Lx,Ly,_,nsamples = np.shape(data_low_param)
data_low_param_new = np.reshape(data_low_param,(Lx,Ly,nsamples))
data_low_param_new = np.reshape(data_low_param_new,(Lx*Ly,nsamples))
data_low_param_new = data_low_param_new.T

data_midlow_param_new = np.reshape(data_midlow_param,(Lx,Ly,nsamples))
data_midlow_param_new = np.reshape(data_midlow_param_new,(Lx*Ly,nsamples))
data_midlow_param_new = data_midlow_param_new.T

data_midhigh_param_new = np.reshape(data_midhigh_param,(Lx,Ly,nsamples))
data_midhigh_param_new = np.reshape(data_midhigh_param_new,(Lx*Ly,nsamples))
data_midhigh_param_new = data_midhigh_param_new.T

data_high_param_new = np.reshape(data_high_param,(Lx,Ly,nsamples))
data_high_param_new = np.reshape(data_high_param_new,(Lx*Ly,nsamples))
data_high_param_new = data_high_param_new.T

In [58]:
print("The average staggered magnetization of the samples in the experimental dataset")
_,_,stagmag_low,_ = calculate_stag_mag(Lx,Ly,data_low_param_new)
_,_,stagmag_midlow,_ = calculate_stag_mag(Lx,Ly,data_midlow_param_new)
_,_,stagmag_midhigh,_ = calculate_stag_mag(Lx,Ly,data_midhigh_param_new)
_,_,stagmag_high,_ = calculate_stag_mag(Lx,Ly,data_high_param_new)
print("delta = -1.545:\n",stagmag_low)
print("delta = 4.455:\n",stagmag_midlow)
print("delta = 4.955:\n",stagmag_midhigh)
print("delta = 13.455:\n",stagmag_high)

The average staggered magnetization of the samples in the experimental dataset
delta = -1.545:
 0.02093359375
delta = 4.455:
 0.049703125
delta = 4.955:
 0.05774609375
delta = 13.455:
 0.17921484375


This is interesting... The noise in the data could be very high...

Check the largest value of delta tested (delta = 15.955)

In [65]:
data_highest_new = np.reshape(data_highest,(Lx,Ly,nsamples))
data_highest_new = np.reshape(data_highest_new,(Lx*Ly,nsamples))
data_highest_new = data_highest_new.T

In [66]:
_,_,stagmag_highest,_ = calculate_stag_mag(Lx,Ly,data_highest_new)
print("staggered magnetization for delta = 15.955:\n",stagmag_highest)

staggered magnetization for delta = 15.955:
 0.18088671875


not much better...

### Maybe try visualizing the data?

In [72]:
print(data_highest[:,:,10])

[[0 1 0 1 1 1 0 1 0 1 0 1 1 0 0 1]
 [1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1]
 [0 1 0 1 0 0 1 0 1 0 1 1 1 0 1 0]
 [1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1]
 [0 1 0 1 0 1 0 0 1 0 1 0 1 1 1 1]
 [1 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1]
 [1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0]
 [1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 0]
 [1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0]
 [1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1]]


maybe the stagmag is extremely sensitive to noise?